In [45]:
import pandas as pd
import numpy as np
import math

In [46]:
event_training = pd.read_csv('training.csv')
event_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test =  pd.read_csv('trocafone_kaggle_test.csv')
paso_5_train = pd.read_csv('train_paso_6.csv')
paso_5_test = pd.read_csv('test_paso_6.csv')

# vamos a realizar un mean encoding segun Pais,region,ciudad, tipo de dispositivo, sistema operativo de busqueda y version del browser

# primero para los usuarios del set de entrenamiento

In [47]:
#primero hago el mean encoding de ciudad, country y region, para eso defino la siguiente funcion:

In [48]:
def encodearTrain(dataframe ,columna , evento , nombre):
    label_count = dataframe.loc[dataframe['event']==evento,[columna,'person','label']]
    count = dataframe.loc[dataframe['event']==evento,[columna,'person']]\
                    .groupby(columna).count().reset_index()
    count.columns=[columna,'count']
    label_count = label_count.groupby([columna,'label']).count().unstack().reset_index().fillna(0)
    label_count.columns=[columna,'label_0_count','label_1_count']
    tabla = label_count.drop('label_0_count',axis=1).merge(count,on=columna,how='inner')
    tabla[nombre] = tabla['label_1_count'] / tabla['count']
    tabla = tabla.drop(['label_1_count','count'],axis=1)
    usuarios_con_evento = tabla.merge(dataframe,on=columna,how='inner').loc[:,['person',nombre]]\
                            .groupby('person').max().reset_index()
    usuarios_sin_evento = dataframe.loc[:,['person','event']].groupby('person')\
                                        .filter(lambda x: (x['event']!=evento).all())\
                                            .reset_index(drop=True).drop_duplicates(subset='person',keep='first')
    usuarios_sin_evento.columns=['person',nombre]
    usuarios_sin_evento[nombre]=-1
    feature = usuarios_sin_evento.merge(usuarios_con_evento,on=['person',nombre],how='outer')
    
    return feature, tabla

In [50]:
mean_city, tabla_city = encodearTrain(event_training,'city','visited site','city_mean')
mean_region, tabla_region = encodearTrain(event_training,'region','visited site','region_mean')
mean_country, tabla_country = encodearTrain(event_training,'country','visited site','country_mean')
mean_device,tabla_device = encodearTrain(event_training,'device_type','visited site','device_mean')
mean_engine, tabla_engine = encodearTrain(event_training,'search_engine','search engine hit','engine_mean')
mean_browser, tabla_browser = encodearTrain(event_training,'browser_version','visited site','browser_mean')
mean_system, tabla_system = encodearTrain(event_training,'operating_system_version','visited site','system_mean')

In [51]:
feature_train = training.merge(mean_region,on='person',how='inner').merge(mean_city,on='person',how='inner')\
                    .merge(mean_country,on='person',how='inner').merge(mean_device,on='person',how='inner')\
                          .merge(mean_engine,on='person',how='inner').merge(mean_system,on='person',how='inner')\
                .merge(paso_5_train,on=['person','label'],how='inner').merge(mean_browser,on='person',how='inner')


In [52]:
feature_train.head()

,person,label,region_mean,city_mean,country_mean,device_mean,engine_mean,system_mean,mean_time_between_events,mean_time_between_co,...,time_to_first_ch,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st,browser_mean
0,0566e9c1,0,0.081024,0.517241,0.088292,0.073837,0.075361,0.318182,4.061890,0.0,...,710451.0,-1.0,-1.0,799392.0,-1.0,-1.0,799533.0,-1.0,631051.0,0.109407
1,6ec7ee77,0,0.085382,0.086865,0.088292,0.073837,-1.000000,0.069285,0.000000,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,336875.0,-1.0,-1.0,0.098707
2,abe7a2fb,0,0.081897,0.096139,0.088292,0.073837,0.075361,0.073882,4.992409,-1.0,...,174045.0,-1.0,-1.0,9509437.0,-1.0,5710043.0,9509437.0,-1.0,-1.0,0.109360
3,34728364,0,0.081897,0.121673,0.088292,0.073837,0.075361,0.222346,4.314423,-1.0,...,-1.0,-1.0,-1.0,526988.0,-1.0,-1.0,1131498.0,-1.0,-1.0,0.109360
4,87ed62de,0,0.111850,0.185022,0.088292,0.102718,-1.000000,0.101844,1.944791,-1.0,...,1124294.0,-1.0,-1.0,-1.0,-1.0,-1.0,1124607.0,-1.0,-1.0,0.034765


# Ahora repito para el set de prediccion, uso las tablas calculadas en la seccion anterior:

In [37]:
# defino la siguiente funcion:

In [53]:
def encodearTest(dataframe, columna,evento,nombre ,tabla):
    
    objeto = dataframe.loc[:,['person',columna]].drop_duplicates(subset=['person',columna],keep='first')
    objeto = objeto.dropna()
    objeto = objeto.merge(tabla,on=columna,how='left')
    objeto['bool'] = objeto[nombre].isnull()
    objeto_uno = objeto.loc[objeto['bool']==True,['person',nombre]]
    objeto_dos = objeto.loc[objeto['bool']==False,['person',nombre]]

    objeto_uno[nombre] = tabla[nombre].sample(n=1)

    objeto_dos = pd.concat([objeto_uno,objeto_dos])
    
    objeto_dos = objeto_dos.groupby('person').max().reset_index()
    usuarios_sin_evento = dataframe.loc[:,['person','event']].groupby('person')\
                                        .filter(lambda x: (x['event']!=evento).all())\
                                            .reset_index(drop=True).drop_duplicates(subset='person',keep='first')
    usuarios_sin_evento.columns=['person',nombre]
    usuarios_sin_evento[nombre]=-1
    
    feature = pd.concat([usuarios_sin_evento,objeto_dos])
    
    return feature

In [54]:
mean_region = encodearTest(event_test,'region','visited site','region_mean',tabla_region)
mean_country = encodearTest(event_test,'country','visited site','country_mean',tabla_country)
mean_device = encodearTest(event_test,'device_type','visited site','device_mean',tabla_device)
mean_engine = encodearTest(event_test,'search_engine','search engine hit','engine_mean',tabla_engine)
mean_browser = encodearTest(event_test,'browser_version','visited site','browser_mean',tabla_browser)
mean_system = encodearTest(event_test,'operating_system_version','visited site','system_mean',tabla_system)
mean_city = encodearTest(event_test,'city','visited site','city_mean',tabla_city)

In [55]:
feature_test = test.merge(mean_region,on='person',how='inner').merge(mean_country,on='person',how='inner')\
                    .merge(mean_engine,on='person',how='inner').merge(mean_city,on='person',how='inner')\
                    .merge(mean_device,on='person',how='inner').merge(paso_5_test,on='person',how='inner')\
                        .merge(mean_browser,on='person',how='inner').merge(mean_system,on='person',how='inner')

In [56]:
feature_test.head()

,person,region_mean,country_mean,engine_mean,city_mean,device_mean,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,...,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st,browser_mean,system_mean
0,4886f805,0.081024,0.088292,0.075361,0.000000,0.073837,2.239299,-1.0,0.000000,0.000000,...,-1.0,-1.0,-1.0,-1.0,1208947.0,1209157.0,-1.0,-1.0,0.109360,0.050162
1,0297fc1e,0.081024,0.088292,-1.000000,0.091503,0.073837,4.325264,-1.0,6.252485,5.104949,...,4198650.0,-1.0,10596743.0,-1.0,12011361.0,12272605.0,-1.0,-1.0,0.109360,0.069285
2,2d681dd8,0.081897,0.088292,0.075361,0.041284,0.102718,4.530487,-1.0,0.000000,5.928233,...,-1.0,-1.0,1209090.0,-1.0,361166.0,1209090.0,-1.0,-1.0,0.078567,0.091081
3,cccea85e,0.081897,0.088292,0.075361,0.046832,0.102718,3.389035,-1.0,0.000000,4.988119,...,-1.0,-1.0,2078596.0,-1.0,40977.0,2078820.0,-1.0,1828920.0,0.078567,0.101844
4,4c8a8b93,0.081897,0.088292,0.075361,0.017794,0.073837,3.130942,-1.0,4.701654,4.260423,...,-1.0,-1.0,1130153.0,-1.0,1207872.0,1208085.0,-1.0,-1.0,0.056415,0.050162


In [57]:
feature_test = feature_test.fillna(0)

In [58]:
feature_train.to_csv('train_paso_7_city.csv',index=False)
feature_test.to_csv('test_paso_7_city.csv',index=False)

In [59]:
feature_test.columns

Index(['person', 'region_mean', 'country_mean', 'engine_mean', 'city_mean',
       'device_mean', 'mean_time_between_events', 'mean_time_between_co',
       'mean_time_between_ch', 'mean_time_between_vs', 'mean_time_between_ad',
       'mean_time_between_bl', 'mean_time_between_st', 'mean_time_between_se',
       'mean_time_between_sp', 'mean_time_between_vp', 'mean_time_between_le',
       'mean_time_between_gl', 'most_frequent_event_0',
       'most_frequent_event_1', 'most_frequent_event_2',
       'most_frequent_event_3', 'most_frequent_event_4',
       'less_frequent_event_0', 'less_frequent_event_1',
       'less_frequent_event_2', 'less_frequent_event_3',
       'less_frequent_event_4', 'count_most_frequent_event',
       'count_less_frequent_event', 'mean_per_event', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing', 'lead',
       'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', '128GB', '16GB